In [ ]:
filenames_csv = "/media1/data/claire/active-clip-behavior/src/filenames_2021_bodies.csv"

## Imports

In [1]:
import keras
import tensorflow as tf

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model
from keras.applications.vgg16 import preprocess_input

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

from PIL import Image
import glob

2023-05-06 11:26:50.043452: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-06 11:26:50.143664: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 11:26:55.606244: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print(tf.__version__)

2.12.0


## Prepare dataset for embedding projector

In [2]:
data_path = "/media1/data/beardata/2021/2021_bodies/2021_singles"

In [3]:
filenames = glob.glob(data_path + '/**/*.JPG', recursive=True)
filenames[:10]

['/media1/data/beardata/2021/2021_bodies/2021_singles/Bearded_Lady/P2030962_0_Bearded_Lady.JPG',
 '/media1/data/beardata/2021/2021_bodies/2021_singles/Bearded_Lady/598A4238_0_Bearded_Lady.JPG',
 '/media1/data/beardata/2021/2021_bodies/2021_singles/Bearded_Lady/P2118441_0_Bearded_Lady.JPG',
 '/media1/data/beardata/2021/2021_bodies/2021_singles/Bearded_Lady/P2030952_0_Bearded_Lady.JPG',
 '/media1/data/beardata/2021/2021_bodies/2021_singles/Bearded_Lady/P1975653_0_Bearded_Lady.JPG',
 '/media1/data/beardata/2021/2021_bodies/2021_singles/Bearded_Lady/P2030950_0_Bearded_Lady.JPG',
 '/media1/data/beardata/2021/2021_bodies/2021_singles/Bearded_Lady/P2063797_0_Bearded_Lady.JPG',
 '/media1/data/beardata/2021/2021_bodies/2021_singles/Bearded_Lady/P2118150_0_Bearded_Lady.JPG',
 '/media1/data/beardata/2021/2021_bodies/2021_singles/Bearded_Lady/P2118445_0_Bearded_Lady.JPG',
 '/media1/data/beardata/2021/2021_bodies/2021_singles/Bearded_Lady/P2030953_0_Bearded_Lady.JPG']

In [10]:
image0 = keras.utils.load_img(filenames[0], target_size=(224,224))
image0 = np.array(image0)
print(image0.shape)

reshaped_image0 = image0.reshape(1,224,224,3)
print(reshaped_image0.shape)

(224, 224, 3)
(1, 224, 224, 3)


In [5]:
x = preprocess_input(reshaped_image0)

In [6]:
def extract_features(filename, model):
    # load the image as a 224x224 array
    img = keras.utils.load_img(filename, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 

    # prepare image for model
    imgx = preprocess_input(reshaped_img)

    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    print(features)
    return features

In [7]:
# load the model first and pass as an argument
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)
#model.save('feature_extractor')

2023-05-06 11:27:24.769551: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2027] Ignoring visible gpu device (device: 1, name: Quadro P620, pci bus id: 0000:b3:00.0, compute capability: 6.1) with core count: 4. The minimum required count is 8. You can adjust this requirement with the env var TF_MIN_GPU_MULTIPROCESSOR_COUNT.
2023-05-06 11:27:25.726198: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9591 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:02:00.0, compute capability: 7.5


In [8]:
features = model.predict(reshaped_image0)
print(features)

2023-05-06 11:27:28.540115: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:429] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2023-05-06 11:27:28.540209: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:438] Possibly insufficient driver version: 515.65.1
2023-05-06 11:27:28.540255: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops_fused_impl.h:625 : UNIMPLEMENTED: DNN library is not found.
2023-05-06 11:27:28.540293: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:GPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): UNIMPLEMENTED: DNN library is not found.
	 [[{{node model/block1_conv1/Relu}}]]


UnimplementedError: Graph execution error:

Detected at node 'model/block1_conv1/Relu' defined at (most recent call last):
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/site-packages/tornado/platform

In [65]:
data = {}

# lop through each image in the dataset
for filename in filenames:
    # try to extract the features and update the dictionary
    feat = extract_features(filename, model)
    data[filename] = feat

data

2023-05-05 18:45:42.138857: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:429] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2023-05-05 18:45:42.138947: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:438] Possibly insufficient driver version: 515.65.1
2023-05-05 18:45:42.138983: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops_fused_impl.h:625 : UNIMPLEMENTED: DNN library is not found.
2023-05-05 18:45:42.139014: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:GPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): UNIMPLEMENTED: DNN library is not found.
	 [[{{node model/block1_conv1/Relu}}]]


UnimplementedError: Graph execution error:

Detected at node 'model/block1_conv1/Relu' defined at (most recent call last):
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/site-packages/tornado/platform

In [55]:
data

{}

In [42]:
# get a list of the filenames
image_names = np.array(list(data.keys()))

# get a list of just the features
image_features = np.array(list(data.values()))
print(image_names)

[]


In [11]:
from keras.applications import VGG16
from keras.utils import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
import numpy as np

# load the pre-trained VGG16 model
model = VGG16(weights='imagenet', include_top=False)

# load an image and preprocess it
img = load_img(filenames[0], target_size=(224, 224))
print(img)
img_arr = img_to_array(img)
img_arr = preprocess_input(img_arr)

# extract the feature vector for the image
features = model.predict(np.array([img_arr]))

# print the shape of the feature vector
print(features.shape)

<PIL.Image.Image image mode=RGB size=224x224 at 0x7F6F8C5689A0>


2023-05-06 11:43:07.957319: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:429] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2023-05-06 11:43:07.957414: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:438] Possibly insufficient driver version: 515.65.1
2023-05-06 11:43:07.957450: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops_fused_impl.h:625 : UNIMPLEMENTED: DNN library is not found.
2023-05-06 11:43:07.957480: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:GPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): UNIMPLEMENTED: DNN library is not found.
	 [[{{node vgg16/block1_conv1/Relu}}]]


UnimplementedError: Graph execution error:

Detected at node 'vgg16/block1_conv1/Relu' defined at (most recent call last):
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/claire/miniconda3/envs/pytorch-behavior/lib/python3.9/site-packages/tornado/platform